In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

In [2]:
#### Chrome ####
c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
c_driver = webdriver.Chrome(options=c_options)

In [3]:
url = "https://www.tesla.com/es_ES/inventory/new/my?arrangeby=plh&range=0"
c_driver.get(url)
time.sleep(1)
# cookie banner: /html/body/div[2]/div[2]/button[2]
boton = c_driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/button[2]')
boton.click()

In [4]:
coches = {
    'modelo': [],
    'descripcion': [],
    'kilometraje': [],
    'precio': [],
    'autonomia': [],
    'velocidad_max': [],
    'time_veloc_max': [],
    'color': [],
    'año': [],
}

In [5]:
def scroll_page(xpath, number):
    seccion = c_driver.find_element(By.XPATH, xpath)
    c_driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    seccion.click()
    time.sleep(2)

    model = c_driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/aside/form/div/div[4]/div/div/div["+str(number)+"]/div/label")
    model = model.text

    
    # scrapping
    c_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    last_height = c_driver.execute_script("return document.body.scrollHeight")  # Altura inicial de la página

    while True:
        # Desplázate hasta el final de la página
        c_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Espera para permitir que el contenido se cargue (si es dinámico)
        
        # Calcula la nueva altura
        new_height = c_driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # Si la altura no cambia, se llegó al final
            break
        last_height = new_height

    time.sleep(2)
    articulos = c_driver.find_elements(By.TAG_NAME, 'article')  # Encuentra todos los artículos
    print(len(articulos))

    return model

In [6]:
def scrap_page(xpath, number, year):
    # locate
    # seccion = c_driver.find_element(By.XPATH, xpath)
    # seccion.click()
    # time.sleep(1)
    name_model = scroll_page(xpath, number)

    # scrapping
    articulos = c_driver.find_elements(By.TAG_NAME, 'article')  # Encuentra todos los artículos
    print(len(articulos))

    for articulo in articulos:
        try:
            time.sleep(1)
            # result-header
            titulo_elemento = articulo.find_element(By.CSS_SELECTOR, "section.result-header div h3 span").text
            descrip1 = articulo.find_element(By.CSS_SELECTOR, "section.result-header div.tds-text_color--10").text
            kilometraje = articulo.find_element(By.CSS_SELECTOR, "section.result-header div div span").text
            precio = articulo.find_element(By.CSS_SELECTOR, "section.result-header div.result-pricing div div span").text
            # result-highlights-cta
            lista = articulo.find_element(By.CSS_SELECTOR, "section.result-highlights-cta div.result-highlights ul")
            elements = lista.find_elements(By.TAG_NAME, 'li')
            autonomia = elements[0].find_element(By.CSS_SELECTOR, 'div span.tds-text--h4').text
            veloc_max = elements[1].find_element(By.CSS_SELECTOR, 'div span.tds-text--h4').text
            time_speed = elements[2].find_element(By.CSS_SELECTOR, 'div span.tds-text--h4').text
            # result-features features-grid
            lista = articulo.find_element(By.CSS_SELECTOR, 'section.result-features.features-grid ul.result-regular-features.tds-list.tds-list--unordered')
            elements = lista.find_elements(By.TAG_NAME, 'li')
            color = elements[0].text

            coches['modelo'].append(name_model)
            coches['descripcion'].append(descrip1)
            coches['kilometraje'].append(kilometraje)
            coches['precio'].append(precio)
            coches['autonomia'].append(autonomia)
            coches['velocidad_max'].append(veloc_max)
            coches['time_veloc_max'].append(time_speed)
            coches['color'].append(color)
            if year != 0:
                coches['año'].append(year)
            else:
                coches['año'].append(titulo_elemento)
            # autonomia = elements[0].text
        except Exception as e:
            print('vacio')


In [7]:
# COCHES NUEVOS
for i in range(1, 5):
    # xpath = f"/html/body/div[1]/div/div[1]/aside/form/div/div[4]/div/div/div[{i}]/input"
    xpath = "/html/body/div[1]/div/div[1]/aside/form/div/div[4]/div/div/div[" + str(i) + "]/input"
    scrap_page(xpath, i, 2024)
    time.sleep(5)

15
15
46
46
24
24
74
74


In [8]:
df_nuevos = pd.DataFrame(coches)
df_nuevos

,modelo,descripcion,kilometraje,precio,autonomia,velocidad_max,time_veloc_max,color,año
0,Model S,"Model S, tracción a las cuatro ruedas",Cuentakilómetros con 6197 km,88.890 €,634,250,"3,2",Azul Oscuro Metalizado,2024
1,Model S,"Model S, tracción a las cuatro ruedas",Cuentakilómetros con 997 km,90.270 €,634,250,"3,2",Azul Oscuro Metalizado,2024
2,Model S,"Model S, tracción a las cuatro ruedas",Cuentakilómetros con 577 km,90.520 €,634,250,"3,2",Negro Sólido,2024
3,Model S,"Model S, tracción a las cuatro ruedas",Cuentakilómetros con 1151 km,91.080 €,634,250,"3,2",Pintura en color Gris Sigilo,2024
4,Model S,"Model S, tracción a las cuatro ruedas",Cuentakilómetros con 27 km,91.790 €,634,250,"3,2",Pintura en color Gris Sigilo,2024
...,...,...,...,...,...,...,...,...,...
154,Model Y,Gran autonomía con tracción integral,Cuentakilómetros con 831 km,59.960 €,533,217,5,Pintura Plateado Mercurio,2024
155,Model Y,Performance con tracción a las cuatro ruedas,60.270 €,60.270 €,514,250,"3,7",Azul Oscuro Metalizado,2024
156,Model Y,Performance con tracción a las cuatro ruedas,Cuentakilómetros con 395 km,61.500 €,514,250,"3,7",Color Negro Sólido,2024
157,Model Y,Performance con tracción a las cuatro ruedas,61.620 €,61.620 €,514,250,"3,7",Color Negro Sólido,2024


In [10]:
# df_nuevos.to_csv('coches_pedro_nuevos.csv', index=False, encoding='utf-8-sig', sep=';')

In [9]:
# COCHES OCASION
seccion = c_driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/aside/form/div/div[5]/div/div/div[2]/input')
c_driver.execute_script("window.scrollTo(0, 0);")
time.sleep(2)
seccion.click()
time.sleep(2)

for i in range(1, 5):
    # xpath = f"/html/body/div[1]/div/div[1]/aside/form/div/div[4]/div/div/div[{i}]/input"
    xpath = "/html/body/div[1]/div/div[1]/aside/form/div/div[4]/div/div/div[" + str(i) + "]/input"
    scrap_page(xpath, i, 0)
    time.sleep(5)

10
10
19
19
5
5
2
2


In [10]:
# coches

In [11]:
df_completo = pd.DataFrame(coches)
df_completo

,modelo,descripcion,kilometraje,precio,autonomia,velocidad_max,time_veloc_max,color,año
0,Model S,"Model S, tracción a las cuatro ruedas",Cuentakilómetros con 6197 km,88.890 €,634,250,"3,2",Azul Oscuro Metalizado,2024
1,Model S,"Model S, tracción a las cuatro ruedas",Cuentakilómetros con 997 km,90.270 €,634,250,"3,2",Azul Oscuro Metalizado,2024
2,Model S,"Model S, tracción a las cuatro ruedas",Cuentakilómetros con 577 km,90.520 €,634,250,"3,2",Negro Sólido,2024
3,Model S,"Model S, tracción a las cuatro ruedas",Cuentakilómetros con 1151 km,91.080 €,634,250,"3,2",Pintura en color Gris Sigilo,2024
4,Model S,"Model S, tracción a las cuatro ruedas",Cuentakilómetros con 27 km,91.790 €,634,250,"3,2",Pintura en color Gris Sigilo,2024
...,...,...,...,...,...,...,...,...,...
190,Model X,Plaid,Cuentakilómetros con 20.074 km,84.000 €,543,262,"2,6",Blanco Perla Multicapas,2023
191,Model X,Plaid,Cuentakilómetros con 15.048 km,84.600 €,543,262,"2,6",Blanco Perla Multicapas,2023
192,Model X,Plaid,Cuentakilómetros con 21.475 km,85.500 €,543,262,"2,6",Plata Medianoche Metalizado,2023
193,Model Y,Model Y Tracción trasera,Cuentakilómetros con 12.146 km,40.500 €,430,217,"6,9",Pintura Plateado Mercurio,2024


In [13]:
c_driver.quit()

In [14]:
import os

In [15]:
path = 'C:/Users/galla/Dropbox/PC/Documents/1 Pedro G Gallardo/Master IA y Big Data/Casa/proyecto tesla/ProyectoTesla/data/raw/tesla.com'

In [16]:
os.chdir(path)

In [17]:
df_completo['pais'] = 'España'

In [18]:
df_completo.to_csv('coches_pedro.csv', index=False, encoding='utf-8-sig', sep=';')